# 4. Pre-Processing and Training Data

In [1]:
#Import required modules
import os
import glob

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import datetime
import matplotlib.dates as mdates

from pathlib import Path

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

## Read Data

In [2]:
# Change directory one step back and save as the root directory
ROOT_DIR = os.path.normpath(os.getcwd() + os.sep + os.pardir)
print(ROOT_DIR)

D:\gitProjects\springboard_capstone_1\Springboard_Capstone_01


In [3]:
#Define file name and location
dataset_csv = 'crude_oil_price_step3_features.csv'
path = '\\data\\interim\\'
f = ROOT_DIR + path + dataset_csv
print(f)

D:\gitProjects\springboard_capstone_1\Springboard_Capstone_01\data\interim\crude_oil_price_step3_features.csv


In [4]:
#cast csv to dataframe
df = pd.read_csv(f)

#convert 'Date' column to date format
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d', errors="raise")

#set Date as index
df.set_index('Date', drop=True, inplace=True)

#print shape of dataframe
print('Shape:',df.shape)

#display df head
display(df.head().T)

Shape: (269, 30)


Date,2000-02-01,2000-03-01,2000-04-01,2000-05-01,2000-06-01
WTI_Price,29.366000,29.842000,25.722000,28.788000,31.822000
Oil_Production_OPEC,26.675516,26.608782,27.516678,28.048591,27.639380
Oil_Production_nonOPEC,46.822508,46.937266,46.618731,46.623961,46.778141
Oil_Production_World,74470.542507,74439.049968,75116.129376,75639.562523,75360.217843
Henry_Hub_NG_Price,2.660000,2.790000,3.040000,3.590000,4.290000
Oil_Production_US,5.851839,5.918207,5.854166,5.846510,5.822882
Petrol_Consumption_OECD,50.229341,49.358083,46.286749,47.338795,47.955820
Petrol_Consumption_nonOECD,28.770679,28.528547,28.131882,28.216862,28.270995
US_CPI,1.700000,1.710000,1.709000,1.712000,1.722000
US_PPI,1.299187,1.305372,1.311693,1.317793,1.323801


## Train/Test Split

In [ ]:
#expected 70-30 split sizes
print(round(len(df) * 0.7, 0))
print(round(len(df) * 0.3, 0))

In [ ]:
#split data to 70$ train and 30% test sets
#... = train_test_split(X, y, test_size, random_state)
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='WTI_Price'),
                                                    df.WTI_Price, test_size=0.3,
                                                    random_state=47)

In [ ]:
print('(X_train, X_test):\n',X_train.shape, X_test.shape)
print('\n(y_train, y_test):\n',y_train.shape, y_test.shape)

## Initial Models

### Imputing missing feature (predictor) values
Impute missing values using scikit-learn. Learn values to impute from a train split and apply that to the test split.

### Impute missing values with median
There's missing values. Recall from your data exploration that many distributions were skewed. Your first thought might be to impute missing values using the median.

In [ ]:
X_defaults_median = X_train.median()
X_defaults_median

In [ ]:
#Call `X_train` and `X_test`'s `fillna()` method, passing `X_defaults_median` as the values to use
#Assign the results to `X_tr` and `X_te`, respectively
X_tr = X_train.fillna(X_defaults_median)
X_te = X_test.fillna(X_defaults_median)

### Scale Data

In [ ]:
def scale_train_test_dataframes(X_tr, X_te):
    '''This function gets a dataframe and normalized columns and return a new data frame'''
    
    #store column names
    names = X_tr.columns
    
    from sklearn.preprocessing import StandardScaler

    #Call the StandardScaler`s fit method on `X_tr` to fit the scaler
    #then use it's `transform()` method to apply the scaling to both the train and test split
    #data (`X_tr` and `X_te`), naming the results `X_tr_scaled` and `X_te_scaled`, respectively

    scaler = StandardScaler()
    
    #scale data using sklearn preprocessing module
    scaler.fit(X_tr)
    
    X_tr_scaled = scaler.transform(X_tr)
    
    X_te_scaled = scaler.transform(X_te)

    #create new df using scaled data
    X_tr_scaled_df = pd.DataFrame(X_tr_scaled, columns=names)
    X_te_scaled_df = pd.DataFrame(X_te_scaled, columns=names)

    #set index
    X_tr_scaled_df.set_index(X_tr.index, drop=True, inplace=True)
    X_te_scaled_df.set_index(X_te.index, drop=True, inplace=True)

    #return scaled dataframe
    return X_tr_scaled_df, X_te_scaled_df

In [ ]:
X_tr_scaled, X_te_scaled = scale_train_test_dataframes(X_tr, X_te)

#### Plot scaled data

In [ ]:
#Plot histogram of all  features
#Call plt.subplots_adjust() with an argument hspace=0.5 to adjust the spacing
#It's important you create legible and easy-to-read plots
X_tr_scaled.hist(figsize=(15,15))
plt.subplots_adjust(hspace=0.5);

In [ ]:
sns.boxplot(data = X_tr_scaled, orient = 'h')

## Conduct Regression Model in PyCaret

In [ ]:
from pycaret.regression import *

# Transform dataset (transform, bin and create dummy variables) and split the dataset.
#In addition, we are logging experiments and plots for those experiment to be viewed later with MLflow. 

reg_model = setup(data=df, 
                target='WTI_Price',
                session_id=786,
                transformation=True,
                normalize=True,
                train_size=0.7,
                log_plots=True)

In [ ]:
# We can do a compare_models() function without assigning it to a variable.
# However, we have top 5 models selected using n_select and assigning it to top5 variable.
# We plan to use this for Stacking and Blending purposes. 
# We have have adjusted the default fold value from 10 to 5.

top5 = compare_models(n_select=5, sort='RMSE', fold=5)

In [ ]:
# We can tune our top 5 models dynamically with a higher iteration rate (n_iter)
#to find more optimal hyper parameters over a larger search space. 

tuned_top5 = [tune_model(i, n_iter=120, optimize='RMSE', fold=5) for i in top5]

## Evaluate best prediction model

In [ ]:
from pycaret.regression import *

In [ ]:
#init setup pycaret.regression
s = setup(X_tr_scaled, target = y_train, session_id = 1)

In [ ]:
#model training and selection
best = compare_models()

In [ ]:
print(best)

In [ ]:
#analyze best model
evaluate_model(best)

In [ ]:
#predict on new data
predictions = predict_model(best, data = X_te_scaled)

In [ ]:
#predictions

In [ ]:
# Blending models is an ensemble method of combining different machine learning algorithms and use a majority vote to build consensus of final prediction values. Let's try building a blending model from our top 5 models and evaluate the results. 
blender_specific = blend_models(estimator_list=tuned_top5[0:], fold=5, optimize='RMSE', choose_better=False)

In [ ]:
# Below is a view of the model parameters. 
print(blender_specific)
display(blender_specific)

In [ ]:
# Stacking models is an ensemble method of using meta learning, 
#where a meta model is created using multiple base estimators to generate the final prediction.
#Let's try building a stacking model from our top 5 models and evaluate the results.

stacker_specific = stack_models(estimator_list=tuned_top5[1:],
                                meta_model=tuned_top5[0],
                                fold=5,
                                optimize='RMSE',
                                choose_better=False)

In [ ]:
# Below is a view of the model parameters. 
print(stacker_specific)
display(stacker_specific)

In [ ]:
evaluate_model(stacker_specific)

In [ ]:
evaluate_model(blender_specific)

In [ ]:
interpret_model(blender_specific)

In [ ]:
pip install shap

In [ ]:
# We can use Pycaret's built in plot_model() function to generate side-by-side plots: 
#the Cook's Distance Outliers and t-SNE Manifold charts. 
fig = plt.figure(figsize=(20,30))
ax = fig.add_subplot(5,2,1)
plot_model(blender_specific, plot='cooks')
ax = fig.add_subplot(5,2,2)
plot_model(blender_specific, plot='manifold')
plt.show()

In [ ]:
# We can use Pycaret's built in plot_model() function to generate side-by-side plots:
#the Residuals chart, Prediction Error and Cross Validation (learning) charts.
#Let's compare the Blend and Stack model plots in a side-by-side comparison. 

fig = plt.figure(figsize=(25,20))
ax = fig.add_subplot(3,2,1)
plot_model(blender_specific, plot='residuals', save=False, verbose=False, scale=0.5)
ax = fig.add_subplot(3,2,2)
plot_model(stacker_specific, plot='residuals', save=False, verbose=False, scale=0.5)
ax = fig.add_subplot(3,2,3)
plot_model(blender_specific, plot='error', save=False, verbose=False, scale=0.5)
ax = fig.add_subplot(3,2,4)
plot_model(stacker_specific, plot='error', save=False, verbose=False, scale=0.5)
ax = fig.add_subplot(3,2,5)
plot_model(blender_specific, plot='learning', verbose=False, scale=0.5)
ax = fig.add_subplot(3,2,6)
plot_model(stacker_specific, plot='learning', save=False, verbose=False, scale=0.5)
#plt.savefig('plots_blender_vs_stacker.png', dpi=300, pad_inches=0.25)
plt.show()

In [ ]:
# We can execute the predict_model() function to use the model to generate the predicted values. 
pred_tunded_blender = predict_model(blender_specific)

In [ ]:
# We can execute the predict_model() function to use the model to generate the predicted values. 
pred_tunded_stacker = predict_model(stacker_specific)

In [ ]:
# The Blend model seems to perform better in both our train and test so let us finalize it.
#The finalize_model() function trains the model on the entire dataset. 

finalize_blender = finalize_model(blender_specific)
display(finalize_blender)

## Analyze the Performance of Final Model on Entire Dataset

In [ ]:
# The predict_model() can be executed with the final blend model over the entire dataset and saved to a csv file. 
pred_final_blender = predict_model(finalize_blender, data=df)
pred_final_blender.to_csv('pred_final_blender.csv')
pred_final_blender.describe().T

In [ ]:
# We can use the Pycaret's built-in plot_model() function to generate Residuals and Error plots for the finalized blend model. 
fig = plt.figure(figsize=(9,10))
ax = fig.add_subplot(2,1,1)
plot_model(finalize_blender, plot='residuals', save=False, verbose=False, scale=0.7)
ax = fig.add_subplot(2,1,2)
plot_model(finalize_blender, plot='error', save=False, verbose=False, scale=0.7)
plt.show()

In [ ]:
# An interesting view is looking at the Actual Values and Predicted Values (Label) in 
#a histogram over the entire dataset. This shows the distribution between the values.
#We can see how the Predicted Values seem to peak in a more distributed manner and skew to the left. 

plt.figure(figsize=(15,5))
sns.set_style("whitegrid")
sns.distplot(pred_final_blender["WTI_Price"],
                bins=20,
                kde=False,
                color="#c6690c")
sns.distplot(pred_final_blender["prediction_label"],
                bins=20,
                kde=False,
                color="#664697")
plt.title("Distribution between Actual Value and Predicted Value (Label)")
plt.ylabel("Count")
plt.xlabel("FCR Value")
plt.legend(('Actual Value', 'Predicted Value (Label)'), ncol=2, loc='upper left', fontsize=12)
plt.show()

In [ ]:
# We can plot the Predicted Value (Label) and Actual Value over the entire dataset. 
sns.regplot(x="WTI_Price", y="prediction_label",
            data=pred_final_blender, lowess=False, 
            scatter_kws ={'s':50}, 
            line_kws={"color": "#664697"},
            color="#c6690c")

plt.title("Linear Relationship between Actual Value and Predicted Value (Label)")
plt.ylabel("Predicted Value (Label)")
plt.xlabel("Actual Value")
plt.legend(('Best Fit', 'Actual Value vs Predicted Value (Label)'), ncol=2, loc='upper left', fontsize=12)
plt.show()

In [ ]:
# We can compare the Predicted Values (Label) and Residuals in an error plot over the entire dataset. 
sns.residplot(x="WTI_Price", y="prediction_label",
              data=pred_final_blender, lowess=False,
              scatter_kws ={'s':50},
              line_kws={"color": "#664697"},
              color="#c6690c")

plt.title("Residuals for the Predicted values in Final Blend Model")
plt.ylabel("Residuals")
plt.xlabel("Predicted Value (Label)")
#plt.xlim((74,101))
plt.legend(('Best Fit', 'Predicted Value (Label)'), ncol=2, loc='upper left', fontsize=12)

In [ ]:
# Sometimes you want to include the output of the compare_models() as a screenshot into a report. However, with the yellow highlights it gets difficult to read. Pycaret has thought of that and you can use the pull() function to show the model results in the sort by or ascending order.
pull().sort_values(by='RMSE', ascending=True)

In [ ]:
# Below is a list of models that Pycaret can use for regression. The ID for each regression can be used to include or exclude models for various functions.
models()